In [1]:
import numpy as np
import pandas as pd
import matplotlib

import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
matplotlib.rc('text', usetex=True)
matplotlib.rcParams.update({"font.size":16})
import os
import sys

sys.path.append(os.path.abspath('../..'))
from modules import io
import modules.vascular_data as sv
from scipy.interpolate import Rbf
import vtk

In [2]:
#CASE = "0110"

#CASE = "0144"

CASE = "coronary"

gen  = 1

FIG_DIR  = "/media/marsdenlab/Data1/UQ/"+CASE+"/figures/3dpca/"+str(gen)+"/"

PCA_DIR = "/media/marsdenlab/Data1/UQ/"+CASE+"/pca/"+str(gen)

BASE_VTP = "/media/marsdenlab/Data1/UQ/"+CASE+"/pca/"+str(gen)+"/mean_groups/exterior.vtp"

io.mkdir(FIG_DIR)

In [3]:
mean_fn = PCA_DIR+"/mean_groups.npy"
mean_groups = np.load(mean_fn)
mean_groups = mean_groups.reshape((-1,3))
print(mean_groups.shape)

V_fn = PCA_DIR+"/V.npy"
V = np.load(V_fn)
V = V.reshape((V.shape[0],-1,3))
print(V.shape)

(1353, 3)
(500, 1353, 3)


In [4]:
vtu = sv.vtk_read_native_polydata(BASE_VTP)
print(vtu)

vtkPolyData (0x5576c7c3eb60)
  Debug: Off
  Modified Time: 510
  Reference Count: 1
  Registered Events: (none)
  Information: 0x5576c7ed71b0
  Data Released: False
  Global Release Data: Off
  UpdateTime: 557
  Field Data:
    Debug: Off
    Modified Time: 298
    Reference Count: 1
    Registered Events: (none)
    Number Of Arrays: 0
    Number Of Components: 0
    Number Of Tuples: 0
  Number Of Points: 29163
  Number Of Cells: 58322
  Cell Data:
    Debug: Off
    Modified Time: 502
    Reference Count: 1
    Registered Events: 
      Registered Observers:
        vtkObserver (0x5576c7f42dc0)
          Event: 33
          EventName: ModifiedEvent
          Command: 0x5576c7edc8b0
          Priority: 0
          Tag: 1
    Number Of Arrays: 3
    Array 0 name = ModelFaceID
    Array 1 name = LocalOpsArray
    Array 2 name = Normals
    Number Of Components: 5
    Number Of Tuples: 58322
    Copy Tuple Flags: ( 1 1 1 1 1 0 1 1 )
    Interpolate Flags: ( 1 1 1 1 1 0 0 1 )
    Pass Th

In [5]:
N  = 20

x = mean_groups[:,0]
y = mean_groups[:,1]
z = mean_groups[:,2]

for i in range(N):
    print(i)
    v  = np.sqrt(np.sum(V[i,:]**2,axis=1))
    
    interp = Rbf(x,y,z,v, function='linear')
    
    array = vtk.vtkDoubleArray()
    array.SetNumberOfComponents(1)
    array.SetNumberOfTuples(vtu.GetNumberOfPoints())
    
    for j in range(vtu.GetNumberOfPoints()):
        point = vtu.GetPoint(j)
        v = interp(*point)
        array.SetValue(j, v)

    vtu.GetPointData().AddArray(array)
    array.SetName(str(i))

sv.vtk_write_native_polydata(vtu,FIG_DIR+'/pca.vtp')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
